In [ ]:
import geopandas as gpd
import pandas as pd

geo_df = gpd.read_file("../data/raw/neighbourhood-boundaries.geojson")

raw_df = pd.read_excel("../data/raw/neighbourhood-profiles-2021-158-model.xlsx")
df = raw_df.transpose().reset_index()
df.columns = ['Neighbourhood'] + list(df.iloc[0, 1:])
df = df[1:]

# data cleaning - removing previous formatting
df['Neighbourhood'] = df['Neighbourhood'].str.strip().str.title()
geo_df['AREA_NAME'] = geo_df['AREA_NAME'].str.replace(r'\s*\(\d+\)', '', regex = True)

merged_data = geo_df.merge(df, left_on = 'AREA_NAME', right_on = 'Neighbourhood', how = 'left')

Duplicate columns: ['    Under $10,000 (including loss)', '    $10,000 to $19,999', '    $20,000 to $29,999', '    $30,000 to $39,999', '    $40,000 to $49,999', '    $50,000 to $59,999', '    $60,000 to $69,999', '    $70,000 to $79,999', '    $80,000 to $89,999', '    $90,000 to $99,999', '    $100,000 and over', '    $10,000 to $19,999', '    $20,000 to $29,999', '    $30,000 to $39,999', '    $40,000 to $49,999', '    $50,000 to $59,999', '    $60,000 to $69,999', '    $70,000 to $79,999', '    $80,000 to $89,999', '    $90,000 to $99,999', '    $100,000 and over', '      $100,000 to $124,999', '      $125,000 and over', '  Market income (%)', '    Employment income (%)', '  Government transfers (%)', '    Employment insurance benefits (%)', '  0 to 17 years', '    0 to 5 years', '  18 to 64 years', '  65 years and over', '  0 to 17 years', '    0 to 5 years', '  18 to 64 years', '  65 years and over', '  0 to 17 years', '    0 to 5 years', '  18 to 64 years', '  65 years and over'

In [60]:
import os

elderly_data = merged_data[[
    "Neighbourhood",
    "  65 years and over - total",
    "  65 years and over - %"
]].copy()

elderly_data.columns = ["Neighbourhood", "65+ Population", "Elderly Distribution"]

cleaned_elderly_data = elderly_data.dropna(axis = 1, how = 'all')

subset = merged_data[[
    "Neighbourhood", "geometry", "  65 years and over - total", "  65 years and over - %"
]]
subset = subset.rename(columns = {
    "  65 years and over - total": "65+ Population",
    "  65 years and over - %": "Elderly Distribution"
})
subset.to_file("../data/elderly_data.geojson", driver = "GeoJSON")